# 문자열 시리즈 데이터 평가와 정제
- 문자열 평가와 대치에 활용하는 메서드
- contains, endswith, findall : 패턴, 문자열 끝의 공백, 복잡한 패턴 찾기에 쓰임
- 대소문자는 upper(), lower()로 구분하여 판단

In [1]:
import pandas as pd
import numpy as np

nls97 = pd.read_csv('data/nls97c.csv')
nls97.set_index('personid', inplace = True)

### 문자열에 패턴 존재 여부 확인
- govprovidejobs(정부에서 일자리를 제공해야 하는가)에 대한 응답에 'not'이 포함되는지를 조사
- contains를 활용

In [2]:
nls97.govprovidejobs.value_counts()

2. Probably          617
3. Probably not      462
1. Definitely        454
4. Definitely not    300
Name: govprovidejobs, dtype: int64

In [4]:
nls97['govprovidejobsdefprob'] = np.where(nls97.govprovidejobs.isnull(), np.nan,
                                          np.where(nls97.govprovidejobs.str.contains('not'), 'No', 'Yes'))

In [5]:
pd.crosstab(nls97.govprovidejobs, nls97.govprovidejobsdefprob)

govprovidejobsdefprob,No,Yes
govprovidejobs,,
1. Definitely,0,454
2. Probably,0,617
3. Probably not,462,0
4. Definitely not,300,0


### 문자열의 시작이나 끝에 있는 공백 처리
- 결혼한 적이 있는 사람(이혼, 사별, 별거 포함)의 시리즈 생성
- married 앞 뒤의 공백에 유의
- startswith, endswith 사용
- strip : 시작과 끝의 공백을 제거(lstrip, rstrip)
---
- 📍다른 시리즈의 값을 바탕으로 시리즈를 생성할 시, 누락값에 주의해야한다.
    - 누락값이 where 호출의 else 조건을 만족시킬 수 있기 때문!

In [6]:
nls97.maritalstatus.value_counts()

Married          3064
Never-married    2766
Divorced          663
Separated         154
Widowed            23
Married             2
Name: maritalstatus, dtype: int64

In [8]:
# 공백으로 시작하는 값이 있는지 검사
nls97.maritalstatus.str.startswith(' ').any()

False

In [9]:
# 공백으로 끝나는 값이 있는지 검사
nls97.maritalstatus.str.endswith(' ').any()

True

In [13]:
nls97['evermarried'] = np.where(nls97.maritalstatus.isnull(), np.nan,
                               np.where(nls97.maritalstatus.str.strip()=='Never-married', 'No', 'Yes'))

In [14]:
pd.crosstab(nls97.maritalstatus, nls97.evermarried)

evermarried,No,Yes
maritalstatus,,
Divorced,0,663
Married,0,3064
Married,0,2
Never-married,2766,0
Separated,0,154
Widowed,0,23


### isin을 사용해 문자열 값이 리스트에 있는지 검사

In [15]:
nls97['receivedba'] = np.where(nls97.highestdegree.isnull(), np.nan,
                              np.where(nls97.highestdegree.str[0:1].isin(['4', '5', '6', '7']), 'Yes', 'No'))

In [16]:
pd.crosstab(nls97.highestdegree, nls97.receivedba)

receivedba,No,Yes
highestdegree,,
0. None,953,0
1. GED,1146,0
2. High School,3667,0
3. Associates,737,0
4. Bachelors,0,1673
5. Masters,0,603
6. PhD,0,54
7. Professional,0,120


### findall을 사용하여 문자열에서 숫잣값 추출
- weeklyhrstv(일주일 동안 tv시청에 보낸 시간) 문자열에 포함된 모든 숫자의 리스트를 생성
- 문자열 속에 숫자만 얻어내기 위해 정규표현식 '\d+'을 전달

In [18]:
pd.concat([nls97.weeklyhrstv.head(),
          nls97.weeklyhrstv.str.findall('\d+').head()], axis=1)

,weeklyhrstv,weeklyhrstv
personid,,
100061,11 to 20 hours a week,"[11, 20]"
100139,3 to 10 hours a week,"[3, 10]"
100284,11 to 20 hours a week,"[11, 20]"
100292,NaN,NaN
100583,3 to 10 hours a week,"[3, 10]"


In [19]:
nls97.weeklyhrstv.str.findall('\d+')

personid
100061    [11, 20]
100139     [3, 10]
100284    [11, 20]
100292         NaN
100583     [3, 10]
            ...   
999291         [2]
999406     [3, 10]
999543     [3, 10]
999698        [40]
999963     [3, 10]
Name: weeklyhrstv, Length: 8984, dtype: object

### findall로 만든 리스트를 사용해 weeklyhrstv 텍스트로부터 숫자 시리즈 생성
- findall로 만든 리스트에서 마지막 원소를 검색하는 함수 정의
- getnum 함수 : 둘 이상의 수가 있는 경우, 두 수의 중간에 가깝도록 숫자 조정

In [20]:
def getnum(numlist):
    highval = 0
    
    if (type(numlist) is list):
        lastval = int(numlist[-1]) #리스트 마지막 값 추출
        if (numlist[0]=='40'):
            highval = 45
        elif (lastval == 2):
            highval = 1
        else:
            highval = lastval - 5
    else:
        highval = np.nan
    return highval

In [21]:
nls97['weeklyhrstvnum'] = nls97.weeklyhrstv.str.findall('\d+').apply(getnum)

In [22]:
pd.crosstab(nls97.weeklyhrstv, nls97.weeklyhrstvnum)

weeklyhrstvnum,1.0,5.0,15.0,25.0,35.0,45.0
weeklyhrstv,,,,,,
11 to 20 hours a week,0,0,1145,0,0,0
21 to 30 hours a week,0,0,0,299,0,0
3 to 10 hours a week,0,3625,0,0,0,0
31 to 40 hours a week,0,0,0,0,116,0
Less than 2 hours per week,1350,0,0,0,0,0
More than 40 hours a week,0,0,0,0,0,176


### 시리즈에 있는 값을 다른 값으로 바꾸기
- weeklyhrscomputer(일주일간 컴퓨터 사용시간) 시리즈는 현재 값으로 정렬이 잘 안됨(문자열 값을 확인)
- 값들을 순서를 나타내는 문자로 변경하여 해결
    - 기존의 값 리스트와 새로 지정하고자 하는 값 리스트를 생성
    - replace 메서드로 기존 값을 새 값으로 교체

In [23]:
comphrsold = ['None', 'Less than 1 hour a week', '1 to 3 hours a week',
             '4 to 6 hours a week', '7 to 9 hours a week', '10 hours or more a week']

comphrsnew = ['A. None', 'B. Less than 1 hour a week', 'C. 1 to 3 hours a week',
             'D. 4 to 6 hours a week', 'E. 7 to 9 hours a week', 'F. 10 hours or more a week']

In [25]:
nls97.weeklyhrscomputer.value_counts().sort_index()

1 to 3 hours a week         733
10 hours or more a week    3669
4 to 6 hours a week         726
7 to 9 hours a week         368
Less than 1 hour a week     296
None                        918
Name: weeklyhrscomputer, dtype: int64

In [26]:
nls97.weeklyhrscomputer.replace(comphrsold, comphrsnew, inplace=True)

In [28]:
nls97.weeklyhrscomputer.value_counts().sort_index()

A. None                        918
B. Less than 1 hour a week     296
C. 1 to 3 hours a week         733
D. 4 to 6 hours a week         726
E. 7 to 9 hours a week         368
F. 10 hours or more a week    3669
Name: weeklyhrscomputer, dtype: int64